In [3]:
import pandas as pd

df1 = pd.read_csv('customers.csv')
df2 = pd.read_csv('transactions.csv')

df = pd.merge(df1, df2, on='customer_id', how='inner') 

df.to_csv('merged_horizontal.csv', index=False)

In [4]:
column_names_index = df.columns
column_names_index

Index(['customer_id', 'acquisition_date', 'segment', 'age', 'city',
       'is_active', 'transaction_id', 'transaction_date', 'product_category',
       'amount', 'year', 'month', 'quarter', 'day_of_week', 'is_weekend',
       'has_discount', 'discount_amount', 'final_amount'],
      dtype='object')

In [5]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [6]:
most_recent_transactions = df.groupby('customer_id')['transaction_date'].max().reset_index()
most_recent_transactions

,customer_id,transaction_date
0,CUST_0002,2022-03-22
1,CUST_0003,2024-12-08
2,CUST_0005,2024-10-20
3,CUST_0006,2024-12-21
4,CUST_0007,2024-11-04
...,...,...
1854,CUST_1995,2024-11-22
1855,CUST_1996,2024-10-25
1856,CUST_1997,2022-07-02
1857,CUST_1998,2024-11-17


In [7]:
from datetime import datetime
df2['transaction_date'] = pd.to_datetime(df2['transaction_date'])

snapshot_date = df['transaction_date'].max() + pd.Timedelta(days=1)

rfm = df.groupby('customer_id').agg({
    'transaction_date': lambda x: (snapshot_date - x.max()).days,  
    'transaction_id': 'count',                                     
    'final_amount': 'sum'                                       
}).reset_index()

rfm.rename(columns={
    'transaction_date': 'Recency',
    'transaction_id': 'Frequency',
    'final_amount': 'Monetary'
}, inplace=True)

rfm.head()


,customer_id,Recency,Frequency,Monetary
0,CUST_0002,1015,5,337.11
1,CUST_0003,23,38,11535.76
2,CUST_0005,72,24,2423.78
3,CUST_0006,10,26,4280.86
4,CUST_0007,57,47,7372.27


In [8]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

most_recent_transactions = df.groupby('customer_id')['transaction_date'].max().reset_index()
most_recent_transactions.rename(columns={'transaction_date': 'last_purchase_date'}, inplace=True)

customers_with_last_purchase = df.merge(
    most_recent_transactions,
    on='customer_id',
    how='left'
)

snapshot_date = df['transaction_date'].max() + pd.Timedelta(days=1)

customers_with_last_purchase['Recency'] = (
    snapshot_date - customers_with_last_purchase['last_purchase_date']
).dt.days

customers_with_last_purchase.head()


,customer_id,acquisition_date,segment,age,city,is_active,transaction_id,transaction_date,product_category,amount,year,month,quarter,day_of_week,is_weekend,has_discount,discount_amount,final_amount,last_purchase_date,Recency
0,CUST_0002,2022-02-20,Medium Value,53,New York,True,TXN_000001,2022-03-01,Sports,111.17,2022,3,Q1,Tuesday,False,False,0.0,111.17,2022-03-22,1015
1,CUST_0002,2022-02-20,Medium Value,53,New York,True,TXN_000002,2022-03-01,Electronics,6.00,2022,3,Q1,Tuesday,False,False,0.0,6.00,2022-03-22,1015
2,CUST_0002,2022-02-20,Medium Value,53,New York,True,TXN_000003,2022-03-01,Home & Garden,31.91,2022,3,Q1,Tuesday,False,False,0.0,31.91,2022-03-22,1015
3,CUST_0002,2022-02-20,Medium Value,53,New York,True,TXN_000004,2022-03-22,Clothing,96.44,2022,3,Q1,Tuesday,False,False,0.0,96.44,2022-03-22,1015
4,CUST_0002,2022-02-20,Medium Value,53,New York,True,TXN_000005,2022-03-22,Clothing,91.59,2022,3,Q1,Tuesday,False,False,0.0,91.59,2022-03-22,1015


In [9]:
popular_products = df['product_category'].value_counts()
popular_products

product_category
Clothing         11726
Electronics       9558
Home & Garden     7839
Books             5960
Sports            3858
Name: count, dtype: int64

In [10]:
weekend_sales = df.groupby('is_weekend')['final_amount'].sum()
weekend_sales

is_weekend
False    5813291.70
True     2335773.19
Name: final_amount, dtype: float64

In [11]:
monthly_trend = df.groupby(['year', 'month']).agg({
    'final_amount': 'sum',
    'transaction_id': 'count'
}).reset_index()

monthly_trend.tail(12)

,year,month,final_amount,transaction_id
24,2024,1,284147.34,1305
25,2024,2,253998.13,1255
26,2024,3,295649.56,1370
27,2024,4,256038.33,1264
28,2024,5,256522.76,1271
29,2024,6,278692.06,1379
30,2024,7,279873.95,1365
31,2024,8,289685.08,1439
32,2024,9,273501.62,1323
33,2024,10,284709.47,1439


In [12]:
top_customers = rfm.sort_values(by='Monetary', ascending=False).head(20)
top_customers

,customer_id,Recency,Frequency,Monetary
1828,CUST_1968,29,99,34578.67
176,CUST_0191,37,78,32408.12
1662,CUST_1790,13,86,32174.58
632,CUST_0679,12,98,30927.73
174,CUST_0189,16,87,30443.35
1166,CUST_1255,9,106,29078.34
1099,CUST_1184,39,74,28497.95
357,CUST_0392,18,81,28085.72
1045,CUST_1125,25,68,27986.88
80,CUST_0088,20,82,27221.88


In [13]:
inactive_customers = rfm[rfm['Recency'] > 180]
inactive_customers.head()

,customer_id,Recency,Frequency,Monetary
0,CUST_0002,1015,5,337.11
7,CUST_0010,203,20,1614.88
8,CUST_0011,202,7,394.06
14,CUST_0018,260,14,936.32
20,CUST_0025,896,3,722.94


In [14]:
transactions_df = df.sort_values(by=['customer_id', 'transaction_date'])
transactions_df['days_between'] = transactions_df.groupby('customer_id')['transaction_date'].diff().dt.days

avg_repurchase_time = transactions_df.groupby('customer_id')['days_between'].mean().dropna()
avg_repurchase_time.head()

customer_id
CUST_0002     5.250000
CUST_0003    12.621622
CUST_0005    27.913043
CUST_0006    32.600000
CUST_0007    17.108696
Name: days_between, dtype: float64

In [15]:
Historical_CLV = df.groupby('customer_id')['amount'].sum().reset_index()
Historical_CLV

,customer_id,amount
0,CUST_0002,337.11
1,CUST_0003,11614.33
2,CUST_0005,2431.15
3,CUST_0006,4280.86
4,CUST_0007,7479.87
...,...,...
1854,CUST_1995,1656.39
1855,CUST_1996,484.00
1856,CUST_1997,674.21
1857,CUST_1998,2534.95


In [16]:
total_revenue = df['final_amount'].sum()
total_customers = df['customer_id'].nunique()

In [17]:
years_active = (
    (df['transaction_date'].max() - df['transaction_date'].min()).days
) / 365

In [18]:
avg_revenue_per_year = total_revenue / years_active
f"Average Revenue per Year: ${avg_revenue_per_year:,.2f}"

'Average Revenue per Year: $2,728,815.31'

In [19]:
customer_lifespans = df.groupby('customer_id').agg({
    'transaction_date': lambda x: (x.max() - x.min()).days / 365
})
average_lifespan = customer_lifespans['transaction_date'].mean()
f"Average Customer Lifespan: {average_lifespan:.2f} years"

'Average Customer Lifespan: 1.38 years'

In [20]:
average_clv = avg_revenue_per_year / total_customers * average_lifespan
f"Estimated Average CLV: ${average_clv:,.2f}"

'Estimated Average CLV: $2,019.87'

In [21]:
rfm['CLV'] = rfm['Monetary']

low_cutoff = rfm['CLV'].quantile(0.2)
high_cutoff = rfm['CLV'].quantile(0.8)

In [22]:
def segment_clv(value):
    if value >= high_cutoff:
        return 'High Value'
    elif value <= low_cutoff:
        return 'Low Value'
    else:
        return 'Medium Value'

rfm['CLV_Segment'] = rfm['CLV'].apply(segment_clv)

In [23]:
rfm[['customer_id', 'CLV', 'CLV_Segment']].head()

,customer_id,CLV,CLV_Segment
0,CUST_0002,337.11,Low Value
1,CUST_0003,11535.76,High Value
2,CUST_0005,2423.78,Medium Value
3,CUST_0006,4280.86,Medium Value
4,CUST_0007,7372.27,High Value


In [24]:
segment_summary = rfm['CLV_Segment'].value_counts(normalize=True) * 100
segment_summary

CLV_Segment
Medium Value    59.978483
Low Value       20.010758
High Value      20.010758
Name: proportion, dtype: float64

In [25]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)

rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

rfm['RFM_Score'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

rfm['RFM_Code'] = (
    rfm['R_Score'].astype(str) +
    rfm['F_Score'].astype(str) +
    rfm['M_Score'].astype(str)
)

rfm.head()

,customer_id,Recency,Frequency,Monetary,CLV,CLV_Segment,R_Score,F_Score,M_Score,RFM_Score,RFM_Code
0,CUST_0002,1015,5,337.11,337.11,Low Value,1,2,1,4,121
1,CUST_0003,23,38,11535.76,11535.76,High Value,4,5,5,14,455
2,CUST_0005,72,24,2423.78,2423.78,Medium Value,3,4,3,10,343
3,CUST_0006,10,26,4280.86,4280.86,Medium Value,5,4,4,13,544
4,CUST_0007,57,47,7372.27,7372.27,High Value,3,5,5,13,355


In [26]:
rfm['RFM_Score'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

def segment_customer(score):
    if score >= 13:
        return 'Champions'
    elif score >= 10:
        return 'Loyal Customers'
    elif score >= 8:
        return 'Potential Loyalists'
    elif score >= 5:
        return 'At Risk'
    else:
        return 'Hibernating'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)

rfm['Segment'].value_counts()

Segment
Loyal Customers        467
Champions              442
Hibernating            353
At Risk                335
Potential Loyalists    262
Name: count, dtype: int64

In [27]:
import pandas as pd

with pd.ExcelWriter('Customer_Insights_Dashboard.xlsx', engine='openpyxl') as writer:
    rfm.to_excel(writer, sheet_name='RFM Analysis', index=False)
    transactions_df.to_excel(writer, sheet_name='Transactions', index=False)

Exception ignored in: <function ZipFile.__del__ at 0x000001E9707207C0>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile.py", line 1894, in __del__
    self.close()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\zipfile.py", line 1911, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
